In [6]:
import yaml

with open(r'unification/unify.yml') as file:
    documents = yaml.full_load(file)
    keys = []
    tables = []
    canonical_id = ''
    for item, doc in documents.items():
        if item == 'keys':
            keys = doc
        if item == 'tables':
            tables = doc
        if item == 'canonical_ids':
            canonical_id = doc[0]['name']

    key_list = []
    for item in keys:
        key_list.append(item['name'])
    
    key_list.append('time')
    key_list.append('ingest_time')
    base_obj = {}
    for key in key_list:
        base_obj[key] = 'null'
    
    table_list = []
    for t in tables:
        tbl = {}
        column = {**base_obj}
        name = t['table']
        tbl['name'] = name
        column['time'] = "time"
        column['ingest_time'] = "TD_TIME_PARSE(cast(CURRENT_TIMESTAMP as varchar))"
        for cl in t['key_columns']:
            col = cl['column']
            key = cl['key']
            column[key] = col

        tbl['columns'] = column
        table_list.append(tbl)

    subque_arr = []
#     q_where =f' where {canonical_id} = t.id\n'
    q_unionall = ' union all\n'
    for table in table_list:
        q_from = ' from ' + table['name'] 
        q_columns = []
        for key, col in table['columns'].items():
            q = f'{col} as {key}'
            q_columns.append(q)
        
        query = 'select ' + ", ".join(q_columns) + q_from
        subque_arr.append(query)
    
    q_subqueries = q_unionall.join(subque_arr)

    r = range(1, len(key_list)+1)
    key_len_arr = ','.join([str(x) for x in [*r]])

    q_part_1 ='''
from
(
'''
#     groupby_str = f') group by {key_len_arr}'

    sql = "SELECT " + ", ".join(key_list) + q_part_1 + q_subqueries + ")"

    print(sql)

SELECT amg_uuid, appnexus, fbp, fingerprint_id, hash_email, mdw_id, permutive_id, plain_text_email, profile_id, source_legacy_id, universal_id, xid, time, ingest_time
from
(
select null as amg_uuid, null as appnexus, null as fbp, null as fingerprint_id, null as hash_email, null as mdw_id, null as permutive_id, null as plain_text_email, null as profile_id, null as source_legacy_id, universal_id as universal_id, null as xid, time as time, TD_TIME_PARSE(cast(CURRENT_TIMESTAMP as varchar)) as ingest_time from cleaned_cds union all
select null as amg_uuid, null as appnexus, null as fbp, null as fingerprint_id, email_md5 as hash_email, null as mdw_id, null as permutive_id, email as plain_text_email, null as profile_id, source_legacy_id as source_legacy_id, null as universal_id, null as xid, time as time, TD_TIME_PARSE(cast(CURRENT_TIMESTAMP as varchar)) as ingest_time from cleaned_cds_athena_data union all
select null as amg_uuid, null as appnexus, null as fbp, null as fingerprint_id, md5_ha

In [2]:
table_list

[{'name': 'cleaned_cds',
  'columns': {'amg_uuid': 'null',
   'appnexus': 'null',
   'fbp': 'null',
   'fingerprint_id': 'null',
   'hash_email': 'null',
   'mdw_id': 'null',
   'permutive_id': 'null',
   'plain_text_email': 'null',
   'profile_id': 'null',
   'source_legacy_id': 'null',
   'universal_id': 'universal_id',
   'xid': 'null',
   'source_table': 'null'}},
 {'name': 'cleaned_cds_athena_data',
  'columns': {'amg_uuid': 'null',
   'appnexus': 'null',
   'fbp': 'null',
   'fingerprint_id': 'null',
   'hash_email': 'email_md5',
   'mdw_id': 'null',
   'permutive_id': 'null',
   'plain_text_email': 'email',
   'profile_id': 'null',
   'source_legacy_id': 'source_legacy_id',
   'universal_id': 'null',
   'xid': 'null',
   'source_table': 'null'}},
 {'name': 'cleaned_cds_alt_email',
  'columns': {'amg_uuid': 'null',
   'appnexus': 'null',
   'fbp': 'null',
   'fingerprint_id': 'null',
   'hash_email': 'md5_hash',
   'mdw_id': 'null',
   'permutive_id': 'null',
   'plain_text_email

In [19]:
import yaml
# import digdag
import pytd.pandas_td as td
import os

def main():
    with open(r'unification/unify.yml') as file:
        documents = yaml.full_load(file)
        keys = []
        tables = []
        canonical_id = ''
        for item, doc in documents.items():
            if item == 'keys':
                keys = doc
            if item == 'tables':
                tables = doc
            if item == 'canonical_ids':
                canonical_id = doc[0]['name']

        key_list = []
        for item in keys:
            key_list.append(item['name'])
        
        key_list.append('time')
        key_list.append('ingest_time')
        base_obj = {}
        for key in key_list:
            base_obj[key] = 'null'
        
        ## GET LIST OF TMP TABLES CREATING IN STG_INC
        ## WE WILL FILTER OUT TABLES THAT HAVE NOT BEEN CREATED
        db = 'information_schema'
        engine = td.create_engine(f'presto:{db}')
        con = td.connect()
        # stg_inc_db = os.environ['stg_inc_db']
        stg_inc_db = 'stg_vishal_inc' 
        # os.environ['TD_API_KEY'] = '12825/23cd76bd54ec169d8b2c9ff44cdc6662a4545992'
        query = f"""SELECT table_name FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_NAME like '%_tmp%' and TABLE_SCHEMA = '{stg_inc_db}'"""

        tmp_tables = td.read_td_query(query, engine)
        tmp_tables =  list(tmp_tables['table_name'])
        
        table_list = []
        for t in tables:
            if (t['table'] + '_tmp') in tmp_tables: 
                tbl = {}
                column = {**base_obj}
                name = t['table']
                tbl['name'] = name
                column['time'] = "time"
                column['ingest_time'] = "TD_TIME_PARSE(cast(CURRENT_TIMESTAMP as varchar))"
                for cl in t['key_columns']:
                    col = cl['column']
                    key = cl['key']
                    column[key] = col

                tbl['columns'] = column
                table_list.append(tbl)

        subque_arr = []
    #     q_where =f' where {canonical_id} = t.id\n'
        q_unionall = ' union all\n'
        for table in table_list:
            q_from = ' from ' + table['name'] 
            q_columns = []
            for key, col in table['columns'].items():
                if key != 'time' and key != 'ingest_time':
                    q = f'CAST({col} as VARCHAR) as {key}'
                else: 
                    q = f'{col} as {key}'
                q_columns.append(q)
            
            query = 'select ' + ", ".join(q_columns) + q_from
            subque_arr.append(query)
        
        q_subqueries = q_unionall.join(subque_arr)

        r = range(1, len(key_list)+1)
        key_len_arr = ','.join([str(x) for x in [*r]])

        q_part_1 ='''
    from
    (
    '''
    #     groupby_str = f') group by {key_len_arr}'

        sql = "SELECT " + ", ".join(key_list) + q_part_1 + q_subqueries + ")"
        # digdag.env.store({'sql' : sql})
        print(sql)

In [28]:
import yaml
# import digdag
import pytd.pandas_td as td
import os

def main():
    with open(r'unification/unify.yml') as file:
        documents = yaml.full_load(file)
        keys = []
        tables = []
        canonical_id = ''
        for item, doc in documents.items():
            if item == 'keys':
                keys = doc
            if item == 'tables':
                tables = doc
            if item == 'canonical_ids':
                canonical_id = doc[0]['name']

        key_list = []
        for item in keys:
            key_list.append(item['name'])
        
        key_list.append('time')
        key_list.append('ingest_time')
        base_obj = {}
        for key in key_list:
            base_obj[key] = 'null'
        
        table_list = []
        for t in tables:
            if t['table'] == current_name:
                tbl = {}
                column = {**base_obj}
                name = t['table']
                tbl['name'] = name
                column['time'] = "time"
                column['ingest_time'] = "TD_TIME_PARSE(cast(CURRENT_TIMESTAMP as varchar))"
                for cl in t['key_columns']:
                    col = cl['column']
                    key = cl['key']
                    column[key] = col

                tbl['columns'] = column
                table_list.append(tbl)

        subque_arr = []
        for table in table_list:
            q_from = ' from ' + table['name'] 
            q_where = f" WHERE TD_TIME_RANGE(inc_unix, {max_date}, TD_TIME_ADD({max_date}, '{time_chunk}'))"
            q_columns = []
            for key, col in table['columns'].items():
                if key != 'time' and key != 'ingest_time':
                    q = f'CAST({col} as VARCHAR) as {key}'
                else: 
                    q = f'{col} as {key}'
                q_columns.append(q)
            
            query = 'select ' + ", ".join(q_columns) + q_from + q_where
            subque_arr.append(query)
        
        sql = subque_arr[0]

        # digdag.env.store({'sql' : sql})
        print(sql)

In [29]:
max_date = 12344
time_chunk = '6w'
current_name = 'cds'
main()

select CAST(null as VARCHAR) as amg_uuid, CAST(null as VARCHAR) as appnexus, CAST(null as VARCHAR) as fbp, CAST(null as VARCHAR) as fingerprint_id, CAST(null as VARCHAR) as hash_email, CAST(null as VARCHAR) as mdw_id, CAST(null as VARCHAR) as permutive_id, CAST(null as VARCHAR) as plain_text_email, CAST(null as VARCHAR) as profile_id, CAST(source_legacy_id as VARCHAR) as source_legacy_id, CAST(universal_id as VARCHAR) as universal_id, CAST(null as VARCHAR) as xid, time as time, TD_TIME_PARSE(cast(CURRENT_TIMESTAMP as varchar)) as ingest_time from cds WHERE TD_TIME_RANGE(inc_unix, 12344, TD_TIME_ADD(12344, '6w'))


In [19]:
stg_inc_db = 'stg_vishal_inc'

['martech_t_chargebee_subscription_created_decrypted_tmp',
 'sailthru_daily_stream_events_tmp',
 'permutive_unify_table_tmp',
 'alterian_xref_email_id_tmp',
 'alterian_xref_universal_id_tmp',
 'sailthru_daily_profile_updates_tmp',
 'martech_d_email_extd_tmp',
 'trackonomics_tmp',
 'daily_email_updates_decrypted_tmp',
 'cds_athena_data_tmp',
 'slv_page_ping_tmp',
 'gam_tmp']